import os
import shutil
from sklearn.model_selection import train_test_split

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [5]:

# Define paths
source_dir = '/Users/selahmitchell/Downloads/alzheimer detection'  # Folder containing all images
train_dir = '/Users/selahmitchell/Downloads/alzheimer detection/train'
test_dir = '/Users/selahmitchell/Downloads/alzheimer detection/test'

# List categories
categories = ['Mild Impairment', 'No Impairment', 'Very Mild Impairment', 'Moderate Impairment']

# Create the directory structure if it doesn't exist
for category in categories:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(test_dir, category), exist_ok=True)

# Loop through each category and split images
for category in categories:
    category_dir = os.path.join(source_dir, category)
    images = [img for img in os.listdir(category_dir) if img.endswith(('.jpg', '.png'))]
    
    # Split data into train and test sets (80% train, 20% test)
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
    
    # Copy images to corresponding train and test folders
    for img in train_images:
        shutil.copy(os.path.join(category_dir, img), os.path.join(train_dir, category, img))
    
    for img in test_images:
        shutil.copy(os.path.join(category_dir, img), os.path.join(test_dir, category, img))

print("Data split complete.")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/selahmitchell/Downloads/alzheimer detection/Mild Impairment'